# Seminar in Statistics

## Personel Project

In [1]:
import os
import json

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
from keras import layers
from keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import Constant
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.python.ops import array_ops
from tqdm import tqdm

from keras import backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
BASE_PATH = '../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/'
TRAIN_DIR = 'stage_2_train/'
TEST_DIR = 'stage_2_test/'

# Load CSV

기존 데이터의 ID를 파일명과 type으로 구분해서 데이터프레임으로 저장

In [3]:
train_df = pd.read_csv(BASE_PATH + 'stage_2_train.csv')
sub_df = pd.read_csv(BASE_PATH + 'stage_2_sample_submission.csv')

train_df['filename'] = train_df['ID'].apply(lambda st: "ID_" + st.split('_')[1] + ".png")
train_df['type'] = train_df['ID'].apply(lambda st: st.split('_')[2])
sub_df['filename'] = sub_df['ID'].apply(lambda st: "ID_" + st.split('_')[1] + ".png")
sub_df['type'] = sub_df['ID'].apply(lambda st: st.split('_')[2])

print(train_df.shape)
train_df.head()

(4516842, 4)


,ID,Label,filename,type
0,ID_12cadc6af_epidural,0,ID_12cadc6af.png,epidural
1,ID_12cadc6af_intraparenchymal,0,ID_12cadc6af.png,intraparenchymal
2,ID_12cadc6af_intraventricular,0,ID_12cadc6af.png,intraventricular
3,ID_12cadc6af_subarachnoid,0,ID_12cadc6af.png,subarachnoid
4,ID_12cadc6af_subdural,0,ID_12cadc6af.png,subdural


In [4]:
test_df = pd.DataFrame(sub_df.filename.unique(), columns=['filename'])
print(test_df.shape)
test_df.head()

(121232, 1)


,filename
0,ID_0fbf6a978.png
1,ID_d62ec3412.png
2,ID_cb544194b.png
3,ID_0d62513ec.png
4,ID_fc45b2151.png


너무 많은 개수의 데이터가 있기 때문에 150000개의 사진을 랜덤으로 선정해 'sample_df' 라는 이름으로 저장

In [5]:
np.random.seed(1000)
sample_files = np.random.choice(os.listdir(BASE_PATH + TRAIN_DIR), 150000)
sample_df = train_df[train_df.filename.apply(lambda x: x.replace('.png', '.dcm')).isin(sample_files)]

In [6]:
sample_files.shape

(150000,)

In [7]:
print(sample_df.shape)
sample_df.head()

(815250, 4)


,ID,Label,filename,type
0,ID_12cadc6af_epidural,0,ID_12cadc6af.png,epidural
1,ID_12cadc6af_intraparenchymal,0,ID_12cadc6af.png,intraparenchymal
2,ID_12cadc6af_intraventricular,0,ID_12cadc6af.png,intraventricular
3,ID_12cadc6af_subarachnoid,0,ID_12cadc6af.png,subarachnoid
4,ID_12cadc6af_subdural,0,ID_12cadc6af.png,subdural


하나의 사진에 여러번의 관측치가 중복되어 있기 때문에, 각 사진별 type을 저장

In [8]:
pivot_df = sample_df[['Label', 'filename', 'type']].drop_duplicates().pivot(
    index='filename', columns='type', values='Label').reset_index()
print(pivot_df.shape)
pivot_df.head()

(135873, 7)


type,filename,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_0000950d7.png,0,0,0,0,0,0
1,ID_0002081b6.png,1,0,1,0,0,0
2,ID_000259ccf.png,0,0,0,0,0,0
3,ID_00027c277.png,0,0,0,0,0,0
4,ID_00027cbb1.png,0,0,0,0,0,0


# Rescale, Resize and Convert to PNG

dcm 확장자로 만들어진 파일의 크기를 조절한 후 png로 다시 저장

In [9]:
def window_image(img, window_center, window_width, intercept, slope, rescale=True):

    img = (img*slope +intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max
    
    if rescale:
        # Extra rescaling to 0-1, not in the original notebook
        img = (img - img_min) / (img_max - img_min)
    
    return img
    
def get_first_of_dicom_field_as_int(x):

    if type(x) == pydicom.multival.MultiValue:
        return int(x[0])
    else:
        return int(x)

def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value, #window center
                    data[('0028','1051')].value, #window width
                    data[('0028','1052')].value, #intercept
                    data[('0028','1053')].value] #slope
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

In [10]:
def save_and_resize(filenames, load_dir):    
    save_dir = '/kaggle/tmp/'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for filename in tqdm(filenames):
        path = load_dir + filename
        new_path = save_dir + filename.replace('.dcm', '.png')
        
        dcm = pydicom.dcmread(path)
        window_center , window_width, intercept, slope = get_windowing(dcm)
        img = dcm.pixel_array
        img = window_image(img, window_center, window_width, intercept, slope)
        
        resized = cv2.resize(img, (224, 224))
        res = cv2.imwrite(new_path, resized)

In [11]:
save_and_resize(filenames=sample_files, load_dir=BASE_PATH + TRAIN_DIR)
save_and_resize(filenames=os.listdir(BASE_PATH + TEST_DIR), load_dir=BASE_PATH + TEST_DIR)

100%|██████████| 121232/121232 [44:58<00:00, 44.93it/s] 


# Data Generator

이미지 데이터의 경우에 사진을 조금씩 움직여 눈에는 비슷하게 보이지만, 실제 데이터는 완전히 다른 데이터를 생성하는 방법을 많이 사용한다.

Keras의 ImageDataGenerator와 flow_from_data 메소드를 사용해 검증용 데이터를 만드는 함수를 정의한다.

In [12]:
BATCH_SIZE = 64

def create_datagen():
    return ImageDataGenerator(validation_split=0.15)

def create_test_gen():
    return ImageDataGenerator().flow_from_dataframe(
        test_df,
        directory='/kaggle/tmp/',
        x_col='filename',
        class_mode=None,
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        shuffle=False
    )

def create_flow(datagen, subset):
    return datagen.flow_from_dataframe(
        pivot_df, 
        directory='/kaggle/tmp/',
        x_col='filename', 
        y_col=['any', 'epidural', 'intraparenchymal', 
               'intraventricular', 'subarachnoid', 'subdural'],
        class_mode='raw',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        subset=subset
    )

In [13]:
data_generator = create_datagen()
train_gen = create_flow(data_generator, 'training')
val_gen = create_flow(data_generator, 'validation')
test_gen = create_test_gen()

Found 115493 validated image filenames.
Found 20380 validated image filenames.
Found 121232 validated image filenames.


# Train Model

### Loss function

불균형한 데이터에서의 성능을 높이기 위해 고안된 Focal Loss Function을 사용한다.

In [14]:
def focal_loss(prediction_tensor, target_tensor, weights=None, alpha=0.25, gamma=2):

    sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)

### DenseNet

DenseNet은 레이어가 많아졌을 때 이전 레이어의 출력값과 거리가 멀어졌을 때 그 정보가 희박해지는 단점을 보완한 딥러닝 기법이다. 

기존 ResNet은 바로 이전의 출력값만을 사용했지만, DenseNet은 해당 레이어 이전에 나왔던 모든 출력값을 사용한다는 차이점이 있다.

합성함수는 Batch Normalization -> ReLU -> 3x3 Conv Layer 로 이어지는 함수이다.

사전학습된 가중치를 적용한 DenseNet 레이어를 사용한다.

In [15]:
densenet = DenseNet121(
    weights='../input/densenet-keras/DenseNet-BC-121-32-no-top.h5',
    include_top=False,
    input_shape=(224,224,3)
)

In [16]:
def build_model():
    model = Sequential()
    model.add(densenet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(6, activation='sigmoid', bias_initializer=Constant(value=-5.5)))
    
    model.compile(
        loss=focal_loss,
        optimizer=Adam(lr=0.001),
        metrics=['accuracy']
    )
    
    return model

In [17]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 6150      
Total params: 7,043,654
Trainable params: 6,960,006
Non-trainable params: 83,648
_________________________________________________________________


In [18]:
checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

total_steps = sample_files.shape[0] / BATCH_SIZE

history = model.fit_generator(
    train_gen,
    steps_per_epoch=2000,
    validation_data=val_gen,
    validation_steps=total_steps * 0.15,
    callbacks=[checkpoint],
    epochs=5
)

Epoch 1/5
2000/2000 [==============================] - 1077s 539ms/step - loss: 0.4124 - accuracy: 0.4455 - val_loss: 0.8615 - val_accuracy: 0.3205
Epoch 2/5
2000/2000 [==============================] - 1036s 518ms/step - loss: 0.2142 - accuracy: 0.5783 - val_loss: 0.0271 - val_accuracy: 0.1366
Epoch 3/5
2000/2000 [==============================] - 1038s 519ms/step - loss: 0.2031 - accuracy: 0.5696 - val_loss: 0.2619 - val_accuracy: 0.5934
Epoch 4/5
2000/2000 [==============================] - 1040s 520ms/step - loss: 0.1884 - accuracy: 0.5564 - val_loss: 0.0662 - val_accuracy: 0.9906
Epoch 5/5
2000/2000 [==============================] - 1039s 520ms/step - loss: 0.1779 - accuracy: 0.5405 - val_loss: 0.6064 - val_accuracy: 1.0000


# Submission

In [20]:
model.load_weights('model.h5')
y_test = model.predict_generator(
    test_gen,
    steps=len(test_gen),
    verbose=1
)

1895/1895 [==============================] - 295s 156ms/step


In [21]:
# Append the output predicts in the wide format to the y_test
test_df = test_df.join(pd.DataFrame(y_test, columns=[
    'any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural'
]))

# Unpivot table, i.e. wide (N x 6) to long format (6N x 1)
test_df = test_df.melt(id_vars=['filename'])

# Combine the filename column with the variable column
test_df['ID'] = test_df.filename.apply(lambda x: x.replace('.png', '')) + '_' + test_df.variable
test_df['Label'] = test_df['value']

test_df[['ID', 'Label']].to_csv('submission.csv', index=False)